In [2]:
import pandas as pd

# 8.1 解析Unix下的时间戳

用pandas来处理Unix下的时间戳并不那么直观——搞清这部分花了我不少时间。接下来我们使用的是我系统中的一个popularity-contest文件。关于这个文件是怎么工作的，可以看[这里](http://popcon.ubuntu.com/README)

但愿这个文件里没啥敏感信息；）

In [3]:
# Read it, and remove the last row
popcon = pd.read_csv('../data/popularity-contest', sep=' ', )[:-1]
popcon.columns = ['atime', 'ctime', 'package-name', 'mru-program', 'tag']

列名包括 access time, created time, package name, recently used program以及一个 tag

In [4]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,NaN
1,1387295796,1354370480,login,/bin/su,NaN
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,NaN
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,NaN


用pandas解析时间戳最神奇的地方在于，numpy的datetimes本身就是以unix的时间戳形式保存的，因此，我们要做的就是，告诉pandas这些int型数据就是时间戳。不需要做其他的改变

首先将这些数据转成int

In [5]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

所有的numpy数组和pandas的series都有一个数据类型dtype，通常是`int64`，`float64`或者`object`。有些时间类型的数据会是`datetime64[s]`或者`datetime64[us]`，此外还有`timedelta`类型。

我们可以使用`pd.to_datetime`函数来完成int型时间戳到datetime的转换。我们并没有改变数据，只是告诉pandas如何去理解这些数据。

In [6]:
popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')

现在再看下数据类型，变成了 `<M8[ns]`据我所知，‘M8’是指 `datetime64`。

In [7]:
popcon['atime'].dtype

dtype('<M8[ns]')

可以看到现在两个实践字段显示成了日期格式

In [8]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,2013-12-17 15:56:37,2013-05-04 02:07:40,perl-base,/usr/bin/perl,NaN
1,2013-12-17 15:56:36,2012-12-01 14:01:20,login,/bin/su,NaN
2,2013-12-17 15:55:43,2012-12-01 05:54:35,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,NaN
3,2013-12-17 15:55:43,2013-12-16 20:03:24,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,2013-12-17 15:55:42,2012-12-01 05:54:13,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,NaN


现在假设我们想看看出了库文件以外的包。

首先，我想把所有时间为0的记录都抛开，注意啦，这里只需要用一个比较符号就可以搞定了，并且！比较的对象就是str类型的！！！

In [9]:
popcon = popcon[popcon['atime'] > '1970-01-01']

接下来使用前面用过的str的contains方法找到不包含lib的包的名字

In [10]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

In [11]:
nonlibraries.sort('ctime', ascending=False)[:10]

,atime,ctime,package-name,mru-program,tag
57,2013-12-17 04:55:39,2013-12-17 04:55:42,ddd,/usr/bin/ddd,<RECENT-CTIME>
450,2013-12-16 20:03:20,2013-12-16 20:05:13,nodejs,/usr/bin/npm,<RECENT-CTIME>
454,2013-12-16 20:03:20,2013-12-16 20:05:04,switchboard-plug-keyboard,/usr/lib/plugs/pantheon/keyboard/options.txt,<RECENT-CTIME>
445,2013-12-16 20:03:20,2013-12-16 20:05:04,thunderbird-locale-en,/usr/lib/thunderbird-addons/extensions/langpac...,<RECENT-CTIME>
396,2013-12-16 20:08:27,2013-12-16 20:05:03,software-center,/usr/sbin/update-software-center,<RECENT-CTIME>
449,2013-12-16 20:03:20,2013-12-16 20:05:00,samba-common-bin,/usr/bin/net.samba3,<RECENT-CTIME>
397,2013-12-16 20:08:25,2013-12-16 20:04:59,postgresql-client-9.1,/usr/lib/postgresql/9.1/bin/psql,<RECENT-CTIME>
398,2013-12-16 20:08:23,2013-12-16 20:04:58,postgresql-9.1,/usr/lib/postgresql/9.1/bin/postmaster,<RECENT-CTIME>
452,2013-12-16 20:03:20,2013-12-16 20:04:55,php5-dev,/usr/include/php5/main/snprintf.h,<RECENT-CTIME>
440,2013-12-16 20:03:20,2013-12-16 20:04:54,php-pear,/usr/share/php/XML/Util.php,<RECENT-CTIME>


它告诉我最近刚使用过ddd，嗯，我清楚滴记得最近确实用过！

本部分想说的重点是，如果你有一个时间段用秒或者微秒来表示的，那么直接转换成`'datetime64[the-right-thing]'`就可以了，剩下的就都交给pandas 了。

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  